In [1]:
import sklearn
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer


/home/ajmalrasi/.virtualenvs/cv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/home/ajmalrasi/.virtualenvs/cv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)


In [2]:
sklearn_version = sklearn.__version__
sklearn_version

'0.21.3'

In [3]:
sentiment_data = pd.read_csv('/home/ajmalrasi/Documents/ML_Deployments_exp/sentimental-analysis-nlp/train_data.csv',
                            header=None,
                            names=['Label','Text'],
                            sep='\t')

In [4]:
sentiment_data.sample(10)

,Label,Text
2659,1,I love Harry Potter.
4834,0,da vinci code sucks...
3160,1,I either LOVE Brokeback Mountain or think it's...
3737,1,Brokeback Mountain was an AWESOME movie.
715,1,DA VINCI CODE IS AWESOME!!
4930,0,The Da Vinci Code sucked big time.
617,1,The Da Vinci Code is awesome..
2650,1,I am going to start reading the Harry Potter s...
5579,0,"Is it just me, or does Harry Potter suck?..."
644,1,da vinci code was an awesome movie...


In [5]:
sentiment_data.shape

(6918, 2)

In [6]:
X = sentiment_data['Text']
Y = sentiment_data['Label']

In [8]:
x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.20)

In [9]:
x_train.shape, x_test.shape

((5534,), (1384,))

In [10]:
tfidf_vect = TfidfVectorizer(max_features=15)

In [11]:
x_trans = tfidf_vect.fit_transform(x_train)

In [14]:
print(x_trans[0:3])

  (0, 3)	0.44731727979214564
  (0, 13)	0.4474423714939744
  (0, 4)	0.4474423714939744
  (0, 12)	0.39814455482020533
  (0, 8)	0.4908959289402105
  (1, 7)	0.759626672459662
  (1, 3)	0.3339197694086178
  (1, 13)	0.3340131496425515
  (1, 4)	0.3340131496425515
  (1, 12)	0.2972126138266706
  (2, 11)	0.5637692557545436
  (2, 5)	0.5637692557545436
  (2, 8)	0.6035962661679877


In [15]:
x_trans.shape

(5534, 15)

In [17]:
clasifier = LinearSVC(C=1.0, max_iter=1000, tol= 0.0001)
linear_svc_model = clasifier.fit(x_trans, y_train)

In [18]:
linear_svc_model

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [19]:
x_test_trans = tfidf_vect.fit_transform(x_test)

In [20]:
x_test_trans.shape

(1384, 15)

In [21]:
y_pred = linear_svc_model.predict(x_test_trans)

In [22]:
y_pred

array([1, 0, 0, ..., 0, 1, 1])

In [23]:
pred_result = pd.DataFrame({'y_test': y_test,
                           'y_pred': y_pred})
pred_result.sample(5)

,y_test,y_pred
3385,1,1
1087,1,1
3425,1,1
115,1,0
6685,0,0


In [24]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.8822254335260116

In [25]:
# Saving the model
text_clf_param = {}
text_clf_param['preprocessing'] = tfidf_vect
text_clf_param['model'] = linear_svc_model
text_clf_param['sklearn_version'] = sklearn_version
text_clf_param['accuracy'] = accuracy

In [26]:
text_clf_param

{'preprocessing': TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.float64'>, encoding='utf-8',
                 input='content', lowercase=True, max_df=1.0, max_features=15,
                 min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                 smooth_idf=True, stop_words=None, strip_accents=None,
                 sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, use_idf=True, vocabulary=None),
 'model': LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
           intercept_scaling=1, loss='squared_hinge', max_iter=1000,
           multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
           verbose=0),
 'sklearn_version': '0.21.3',
 'accuracy': 0.8822254335260116}

In [27]:
import joblib

In [28]:
filename = '/home/ajmalrasi/Documents/ML_Deployments_exp/sentimental-analysis-nlp/text_clf_checkpoint.joblib'

In [29]:
joblib.dump(text_clf_param, filename)

['/home/ajmalrasi/Documents/ML_Deployments_exp/sentimental-analysis-nlp/text_clf_checkpoint.joblib']

In [30]:
# deserializing
clf_checkpoint = joblib.load(filename)

In [31]:
reloaded_vect = clf_checkpoint['preprocessing']
reloaded_vect

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=15,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [32]:
clf_model = clf_checkpoint['model']

In [33]:
clf_model

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [34]:
x_test_trans_new = reloaded_vect.fit_transform(x_test)

In [35]:
y_pred = clf_model.predict(x_test_trans_new)

In [36]:
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.8822254335260116

# pipeline

In [37]:
from sklearn.pipeline import Pipeline

In [38]:
clf_pipeline = Pipeline(steps=[('tfidf_vect', tfidf_vect), ('classifier', clasifier)])
pipeline_model = clf_pipeline.fit(x_train, y_train)

In [39]:
y_pred = pipeline_model.predict(x_test)

In [40]:
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.8822254335260116

In [41]:
pipe_clf_prams = {}

pipe_clf_prams['pipeline_clf'] = pipeline_model
pipe_clf_prams['sklearn_version'] = sklearn_version
pipe_clf_prams['accuracy'] = accuracy

In [42]:
import joblib


In [44]:
filename = '/home/ajmalrasi/Documents/ML_Deployments_exp/sentimental-analysis-nlp/pipe_clf_checkpoint.joblib'

In [45]:
joblib.dump(pipe_clf_prams,filename)

['/home/ajmalrasi/Documents/ML_Deployments_exp/sentimental-analysis-nlp/pipe_clf_checkpoint.joblib']

In [46]:
pipe_clf_ckeckpoint = joblib.load(filename)

In [47]:
reloaded_pipeline = pipe_clf_ckeckpoint['pipeline_clf']

reloaded_pipeline

Pipeline(memory=None,
         steps=[('tfidf_vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=15,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('classifier',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
     

In [48]:
y_pred = reloaded_pipeline.predict(x_test)

In [49]:
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.8822254335260116